# Construction Loan Estimator

## Parameters

In [71]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from dateutil.relativedelta import relativedelta
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
from math import e

output_notebook()

Loading BokehJS ...

In [28]:
property_cost = 85000
construction_cost = 115000
loan_term = 15
loan_term_months = loan_term * 12
completed_value = 300000
construction_term = 12
down_payment = 40000
inflation = 0.02
closing_costs = 0.05
square_footage = 1750
maintenance_per_sqft = 0.75
assessment_rate = 0.07
mill_levy = 95
initial_rent = 1500
rent_growth_rate = 0.02
rent_growth_rate_monthly = rent_growth_rate / 12
home_appreciation = 0.05
home_appreciation_month = home_appreciation / 12
current_month = datetime.date.today().replace(day=1)
total_loan_amount = property_cost + construction_cost - down_payment
loan_interest_rate = 0.04
loan_rate_monthly = loan_interest_rate / 12
investment_growth_rate = 0.06
investment_growth_rate_monthly = investment_growth_rate / 12

In [47]:
# Create blank table for monthly level data
monthly_data = pd.DataFrame()

# Create Period to offset the index by one, to indicate month and create periods
monthly_data['Overall Month'] = range(1, loan_term_months + construction_term + 2)
monthly_data['Overall Year'] = ((monthly_data['Overall Month'] - 1) // 12) + 1
monthly_data['Year Start'] = np.where((((monthly_data['Overall Month'] - 1) // 12) == (monthly_data['Overall Month'] - 1) / 12) | (monthly_data['Overall Month'] == 1), 1, 0)
monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= construction_term, monthly_data['Overall Month'], 0)
monthly_data['Loan Month'] = np.where(monthly_data['Overall Month'] > construction_term, monthly_data['Overall Month'] - construction_term, 0)

# Calculate home value
monthly_data['Home Value'] = np.where(monthly_data['Overall Month'] <= construction_term, 0, completed_value * (e**(home_appreciation_month * (monthly_data['Loan Month']))))

# Calculate loan value
monthly_data['Loan Value'] = np.where(monthly_data['Loan Month'] != 0, total_loan_amount * (((1 + loan_rate_monthly) ** (loan_term_months)) - ((1 + loan_rate_monthly)**(monthly_data['Loan Month'] - 1))) / ((1 + loan_rate_monthly)**(loan_term_months) - 1), total_loan_amount)

# Calculate disbursement payments in a linear fashion
monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= construction_term, total_loan_amount / construction_term, 0)

# Calculate monthly payments
monthly_data['Mortgage Payment'] = np.where(monthly_data['Overall Month'] > construction_term, total_loan_amount * (loan_rate_monthly * ((1 + loan_rate_monthly)**loan_term_months)) / ((1 + loan_rate_monthly)**loan_term_months - 1), 0)

# Calculate interest payment
monthly_data['Interest Payment'] = np.where(monthly_data['Overall Month'] <= construction_term, monthly_data['Construction Disbursements'].cumsum() * loan_rate_monthly, monthly_data['Loan Value'] * loan_rate_monthly)
monthly_data['Cumulative Interest Payments'] = monthly_data['Interest Payment'].cumsum()

# Calculate principal payments
monthly_data['Principal Payment'] = np.where(monthly_data['Overall Month'] > construction_term, monthly_data['Mortgage Payment'] - monthly_data['Interest Payment'], 0)
monthly_data['Cumulative Principal Payments'] = monthly_data['Principal Payment'].cumsum()

# Calculate property taxes
monthly_data['Property Tax Payment'] = np.where(monthly_data['Overall Month'] > construction_term, (((monthly_data['Home Value'] * assessment_rate) * (mill_levy / 1000)) / 12), 0)
monthly_data['Cumulative Property Taxes'] = monthly_data['Property Tax Payment'].cumsum()

# Calculate maintenance costs
monthly_data['Maintenance Costs'] = np.where(monthly_data['Overall Month'] > construction_term, ((square_footage * maintenance_per_sqft) / 12), 0)
monthly_data['Cumulative Maintenance Costs'] = monthly_data['Maintenance Costs'].cumsum()

# Calculate total home payments
monthly_data['Total Home Payments'] = monthly_data['Interest Payment'] + monthly_data['Principal Payment'] + monthly_data['Property Tax Payment'] + monthly_data['Maintenance Costs']
monthly_data['Cumulative Home Payments'] = monthly_data['Total Home Payments'].cumsum() + down_payment

# Calculate equity metrics
monthly_data['Equity Ownership Proportion'] = 1- (monthly_data['Loan Value'] / total_loan_amount)
monthly_data['Equity Value'] = monthly_data['Equity Ownership Proportion'] * monthly_data['Home Value']
monthly_data['Equity Gain'] = monthly_data['Equity Value'] - monthly_data['Cumulative Home Payments']

# Calculate rent payments
monthly_data['Rent Payments'] = initial_rent * (e**(rent_growth_rate_monthly * (monthly_data['Overall Month'] - 1)))
monthly_data['Cumulative Rent Payments'] = monthly_data['Rent Payments'].cumsum()

# Calculate rent investment value
monthly_data['Rent Investment'] = down_payment * (e**(investment_growth_rate_monthly * (monthly_data['Overall Month'] - 1)))

# Calculate when intersection points
monthly_data['Decision'] = np.where(monthly_data['Cumulative Rent Payments'] - monthly_data['Cumulative Home Payments'] > 0, 'Buy', 'Rent')

monthly_data.head(24)

,Overall Month,Overall Year,Year Start,Construction Month,Loan Month,Home Value,Loan Value,Construction Disbursements,Mortgage Payment,Interest Payment,...,Cumulative Maintenance Costs,Total Home Payments,Cumulative Home Payments,Equity Ownership Proportion,Equity Value,Equity Gain,Rent Payments,Cumulative Rent Payments,Rent Investment,Decision
0,1,1,1,1,0,0.000000,160000.000000,13333.333333,0.000000,44.444444,...,0.000,44.444444,40044.444444,0.000000,0.000000,-40044.444444,1500.000000,1500.000000,40000.000000,Rent
1,2,1,0,2,0,0.000000,160000.000000,13333.333333,0.000000,88.888889,...,0.000,88.888889,40133.333333,0.000000,0.000000,-40133.333333,1502.502084,3002.502084,40200.500834,Rent
2,3,1,0,3,0,0.000000,160000.000000,13333.333333,0.000000,133.333333,...,0.000,133.333333,40266.666667,0.000000,0.000000,-40266.666667,1505.008343,4507.510427,40402.006683,Rent
3,4,1,0,4,0,0.000000,160000.000000,13333.333333,0.000000,177.777778,...,0.000,177.777778,40444.444444,0.000000,0.000000,-40444.444444,1507.518781,6015.029208,40604.522585,Rent
4,5,1,0,5,0,0.000000,160000.000000,13333.333333,0.000000,222.222222,...,0.000,222.222222,40666.666667,0.000000,0.000000,-40666.666667,1510.033408,7525.062616,40808.053601,Rent
5,6,1,0,6,0,0.000000,160000.000000,13333.333333,0.000000,266.666667,...,0.000,266.666667,40933.333333,0.000000,0.000000,-40933.333333,1512.552228,9037.614844,41012.604821,Rent
6,7,1,0,7,0,0.000000,160000.000000,13333.333333,0.000000,311.111111,...,0.000,311.111111,41244.444444,0.000000,0.000000,-41244.444444,1515.075251,10552.690095,41218.181358,Rent
7,8,1,0,8,0,0.000000,160000.000000,13333.333333,0.000000,355.555556,...,0.000,355.555556,41600.000000,0.000000,0.000000,-41600.000000,1517.602481,12070.292576,41424.788352,Rent
8,9,1,0,9,0,0.000000,160000.000000,13333.333333,0.000000,400.000000,...,0.000,400.000000,42000.000000,0.000000,0.000000,-42000.000000,1520.133928,13590.426504,41632.430968,Rent
9,10,1,0,10,0,0.000000,160000.000000,13333.333333,0.000000,444.444444,...,0.000,444.444444,42444.444444,0.000000,0.000000,-42444.444444,1522.669597,15113.096101,41841.114396,Rent


In [55]:
# Shrink set to year starts for simpler graphs
year_data = monthly_data.loc[monthly_data['Year Start'] == 1]
year_data.head(10)

,Overall Month,Overall Year,Year Start,Construction Month,Loan Month,Home Value,Loan Value,Construction Disbursements,Mortgage Payment,Interest Payment,...,Cumulative Maintenance Costs,Total Home Payments,Cumulative Home Payments,Equity Ownership Proportion,Equity Value,Equity Gain,Rent Payments,Cumulative Rent Payments,Rent Investment,Decision
0,1.0,1.0,1.0,1.0,0.0,0.000000,160000.000000,13333.333333,0.000000,44.444444,...,0.000,44.444444,40044.444444,0.000000,0.000000,-40044.444444,1500.000000,1500.000000,40000.000000,Rent
12,13.0,2.0,1.0,0.0,1.0,301252.607787,160000.000000,0.000000,1183.500681,533.333333,...,109.375,1459.819834,44926.486501,0.000000,0.000000,-44926.486501,1530.302010,19696.361238,42473.461862,Rent
24,25.0,3.0,1.0,0.0,13.0,316698.159275,152053.353730,0.000000,1183.500681,506.844512,...,1421.875,1468.379244,62499.535698,0.049667,15729.301538,-46770.234160,1561.216161,38260.313356,45099.874063,Rent
36,37.0,4.0,1.0,0.0,25.0,332935.621121,143782.948831,0.000000,1183.500681,479.276496,...,2734.375,1477.377504,80178.128549,0.101357,33745.212524,-46432.916025,1592.754820,57199.282183,47888.694525,Rent
48,49.0,5.0,1.0,0.0,37.0,350005.595439,135175.594875,0.000000,1183.500681,450.585316,...,4046.875,1486.837115,97967.676395,0.155153,54304.254358,-43663.422037,1624.930602,76520.843560,50849.966013,Rent
60,61.0,6.0,1.0,0.0,49.0,367950.766054,126217.564038,0.000000,1183.500681,420.725213,...,5359.375,1496.781730,115873.868019,0.211140,77689.207446,-38184.660573,1657.756377,96232.726367,53994.352303,Rent
72,73.0,7.0,1.0,0.0,61.0,386816.005242,116894.569204,0.000000,1183.500681,389.648564,...,6671.875,1507.236217,133902.683877,0.269409,104211.690906,-29690.992971,1691.245277,116342.815622,57333.176582,Rent
84,85.0,8.0,1.0,0.0,73.0,406648.485927,107191.741175,0.000000,1183.500681,357.305804,...,7984.375,1518.226717,152060.411045,0.330052,134214.990597,-17845.420448,1725.410698,136859.155628,60878.462225,Rent
96,97.0,9.0,1.0,0.0,85.0,427497.799640,97093.604962,0.000000,1183.500681,323.645350,...,9296.875,1529.780712,170353.658950,0.393165,168077.159138,-2276.499812,1760.266306,157789.953194,64642.976088,Rent
108,109.0,10.0,1.0,0.0,97.0,449416.080526,86584.055099,0.000000,1183.500681,288.613517,...,10609.375,1541.927092,188789.375887,0.458850,206214.413785,17425.037897,1795.826045,179143.580919,68640.274487,Rent


In [82]:
# Plot payment comparison
payments = figure(title='Rent vs. Buy Cumulative Cost')
payments.xaxis.axis_label = "Year"
payments.yaxis.formatter.use_scientific = False
payments.yaxis.axis_label = "Cost"

payments.line(year_data['Overall Year'], year_data['Cumulative Rent Payments'], line_color='red')
payments.line(year_data['Overall Year'], year_data['Cumulative Home Payments'], line_color='blue')
show(payments)

In [83]:
# Plot returns comparison
payments = figure(title='Rent vs. Buy Cumulative Returns')
payments.xaxis.axis_label = "Year"
payments.yaxis.formatter.use_scientific = False
payments.yaxis.axis_label = "Returns"

payments.line(year_data['Overall Year'], year_data['Equity Gain'], line_color='red')
payments.line(year_data['Overall Year'], year_data['Rent Investment'], line_color='blue')
show(payments)